<a href="https://colab.research.google.com/github/pamelag/level-up-ai/blob/main/Faster_LLM_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!pip install pip
!pip install bitsandbytes
!pip install torch
!pip install transformers
!pip install accelerate
!pip install peft
!pip install datasets
!pip install loralib
!pip install einops

In [ ]:
import torch
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b", padding_side="left")
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
DEVICE = "cuda:0"

prompt = f"""
: Digital Products Digital Taxes is working on developing a roadmap for customer identity requirements as part of Single identity for Customers Aspire Initiative under Digital and Customer Identity strategic Program and passionate customer focus strategic Focus Area. The initiative lead of Digital Identity is Dan Papallo, Director, Digital Products Digital Taxes. High Level Actions under this initiative is Understand and develop a roadmap for customer identity requirements across all of Revenue NSW. Major Milestones - Define the unified and holistic view of Revenue NSW customer identity across multiple touchpoints and products, Develop an implementation roadmap,Complete implementation of Revenue Identity Management Service (phase 1), Develop and release Property Data Service proof of concept.Develop customer grouping initial solution.
:
""".strip()


encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [ ]:
config = PeftConfig.from_pretrained("trained-model")
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, "trained-model")

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 250
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%timeit -r 5

with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
        do_sample=False,
        use_cache=True,
    )

In [ ]:
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
        do_sample=False,
        use_cache=True,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_use_double_quant=True,
    bnb_8bit_quant_type="nf8",
    bnb_8bit_compute_dtype=torch.bfloat16,
)

In [ ]:
config = PeftConfig.from_pretrained("trained-model")
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    load_in_8bit_fp32_cpu_offload=True,
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, "trained-model")

In [ ]:
config = PeftConfig.from_pretrained("trained-model")

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    device_map="auto",
    trust_remote_code=True,
    load_in_8bit=True,
)

model = PeftModel.from_pretrained(model, MODEL_ID)